#### Environment Setup

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import math

#### Matching Sub-Districts to nearby Soil Stations and process it in a final form

In [2]:
soil_locs = pd.read_csv(r'C:\Users\Manan Arora\Desktop\AgriTech Project\Data Driven Business Metrics Project\Data\Soil Data\AllSoilLocations-Geocoordinates.csv')

In [3]:
ph_data = pd.read_csv(r'C:\Users\Manan Arora\Desktop\AgriTech Project\Data Driven Business Metrics Project\Data\Soil Data\SoilpHValues_Aggregated.csv')
macro_data = pd.read_csv(r'C:\Users\Manan Arora\Desktop\AgriTech Project\Data Driven Business Metrics Project\Data\Soil Data\SoilMacroNutrients_Aggregated.csv').drop('Unnamed: 0',axis=1)
micro_data = pd.read_csv(r'C:\Users\Manan Arora\Desktop\AgriTech Project\Data Driven Business Metrics Project\Data\Soil Data\SoilMicroNutrients_Aggregated.csv')
ph_data['Address'] = ph_data['Sub District'] + ', ' + ph_data['District'] + ', ' +  ph_data['State']
macro_data['Address'] = macro_data['Sub-District'] + ', ' + macro_data['District'] + ', ' + macro_data['State']
micro_data['Address'] = micro_data['Sub-District'] + ', ' + micro_data['District'] + ', ' + micro_data['State']

In [4]:
macro_data_loc = macro_data.merge(soil_locs,on='Address').reset_index().rename(columns={'index':'SoilNStation'})
ph_data_loc = ph_data.merge(soil_locs,on='Address').reset_index().rename(columns={'index':'SoilpHStation'})
micro_data_loc = micro_data.merge(soil_locs,on='Address').reset_index().rename(columns={'index':'SoilMStation'})

In [5]:
macro_data_loc.to_csv(r'C:\Users\Manan Arora\Desktop\AgriTech Project\Data Driven Business Metrics Project\Data\Soil Data\SoilMacroNutrients_Geo.csv',index=False)
micro_data_loc.to_csv(r'C:\Users\Manan Arora\Desktop\AgriTech Project\Data Driven Business Metrics Project\Data\Soil Data\SoilMicroNutrients_Geo.csv',index=False)
ph_data_loc.to_csv(r'C:\Users\Manan Arora\Desktop\AgriTech Project\Data Driven Business Metrics Project\Data\Soil Data\SoilpHValues_Geo.csv',index=False)

In [6]:
subdist_data = pd.read_csv(r'C:\Users\Manan Arora\Desktop\AgriTech Project\Data Driven Business Metrics Project\FinalSubDistData\Geolocation-SubDistricts.csv')

In [7]:
def distance(df):
    lat1, lon1 = df['lat'], df['long']
    distances = []
    for i in range(len(macro_data_loc)):
        lat2, lon2 = macro_data_loc['Latititude'][i],macro_data_loc['Longitude'][i]
        radius = 6371 # km
        dlat = math.radians(lat2-lat1)
        dlon = math.radians(lon2-lon1)
        a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
            * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        d = radius * c
        distances.append(d)
    
    stationno = macro_data_loc['SoilNStation'][np.array(distances).argmin()]

    return stationno

In [8]:
subdist_data['SoilNStation'] = subdist_data.apply(distance,axis=1)
subdist_data = subdist_data.merge(macro_data_loc.drop(['State','District','Sub-District','Latititude','Longitude','Address'],axis=1),on='SoilNStation').drop('SoilNStation',axis=1)

In [9]:
def distance(df):
    lat1, lon1 = df['lat'], df['long']
    distances = []
    for i in range(len(micro_data_loc)):
        lat2, lon2 = micro_data_loc['Latititude'][i],micro_data_loc['Longitude'][i]
        radius = 6371 # km
        dlat = math.radians(lat2-lat1)
        dlon = math.radians(lon2-lon1)
        a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
            * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        d = radius * c
        distances.append(d)
    
    stationno = micro_data_loc['SoilMStation'][np.array(distances).argmin()]

    return stationno

In [10]:
subdist_data['SoilMStation'] = subdist_data.apply(distance,axis=1)
subdist_data = subdist_data.merge(micro_data_loc.drop(['State','District','Sub-District','Latititude','Longitude','Address'],axis=1),on='SoilMStation').drop('SoilMStation',axis=1)

In [11]:
def distance(df):
    lat1, lon1 = df['lat'], df['long']
    distances = []
    for i in range(len(ph_data_loc)):
        lat2, lon2 = ph_data_loc['Latititude'][i],ph_data_loc['Longitude'][i]
        radius = 6371 # km
        dlat = math.radians(lat2-lat1)
        dlon = math.radians(lon2-lon1)
        a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
            * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        d = radius * c
        distances.append(d)
    
    stationno = ph_data_loc['SoilpHStation'][np.array(distances).argmin()]

    return stationno

In [12]:
subdist_data['SoilpHStation'] = subdist_data.apply(distance,axis=1)
subdist_data = subdist_data.merge(ph_data_loc.drop(['State','District','Sub District','Latititude','Longitude','Address'],axis=1),on='SoilpHStation').drop('SoilpHStation',axis=1)

In [ ]:
subdist_data['N_D'] = subdist_data['N_D'].apply(lambda x: x[0:-1])
subdist_data['P_D'] = subdist_data['P_D'].apply(lambda x: x[0:-1])
subdist_data['K_D'] = subdist_data['K_D'].apply(lambda x: x[0:-1])
subdist_data['Zn_D'] = subdist_data['Zn_D'].apply(lambda x: x[0:-1])
subdist_data['Fe_D'] = subdist_data['Fe_D'].apply(lambda x: x[0:-1])
subdist_data['Cu_D'] = subdist_data['Cu_D'].apply(lambda x: x[0:-1])
subdist_data['Mn_D'] = subdist_data['Mn_D'].apply(lambda x: x[0:-1])
subdist_data['B_D'] = subdist_data['B_D'].apply(lambda x: x[0:-1])
subdist_data['N_D'] = subdist_data['N_D'].astype(float)
subdist_data['P_D'] = subdist_data['P_D'].astype(float)
subdist_data['K_D'] = subdist_data['K_D'].astype(float)

In [25]:
subdist_data['OC_D'] = subdist_data['OC_D'].apply(lambda x: x[0:-1])

In [28]:
subdist_data = subdist_data.drop(['Zn_S','Fe_S','Cu_S','Mn_S','B_S'],axis=1)

In [37]:
subdist_data.fillna(0).to_csv(r'C:\Users\Manan Arora\Desktop\AgriTech Project\Data Driven Business Metrics Project\Data\Soil Data\Soil_SubDist_AllData_New.csv',index=False)